# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Входные данные от банка — статистика о платёжеспособности клиентов.

**Цель исследования**

Нужно ответить на вопросы, как влияет на факт погашения кредита в срок следующие факторы:
* колиечество детей
* семейное положение
* уровень дохода
* цели кредита

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Ход исследования**

 1. Обзор данных.
 2. Предобработка данных.
 3. Ответы на вопросы.


### Шаг 1. Обзор данных

Импортируем библиотеку Pandas.

In [1]:
import pandas as pd # импорт библиотеки Pandas

Сохраним датасет в переменную df. Чтобы ознакомиться с таблицей, выведем на экран первые 10 строк.

In [2]:
df = pd.read_csv('/datasets/data.csv') # сохранение датасета в переменную
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Изучим общую информацию о данных с помощью методов info и describe.

In [3]:
df.info() # обзор данных
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В таблице 12 столбцов.

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В столбцах `days_employ` и `total_income` есть пропущенные значения. Тип обоих столбцов - float64.

### Шаг 2.1 Проверка данных на аномалии и исправления.

В первую очередь избавимся от аномалий в данных, а после этого заполним пропуски. При поверхностном изучении датасета были обнаружены следующие аномалии:
* Отрицательное количество дней трудового стажа в столбце `days_employed`
* Огромные значения трудового стажа (например значение 340266 соответствует 932 годам, что явялется невозможным)

Изучим датасет более детально, возможно в других числовых ячейках с количественными переменными тоже содержатся отрицательные значения. Проверим каждый столбец по отдельности.

In [4]:
print('Отрицательных значения в столбце children:', df[df['children'] < 0]['children'].count()) # счетчик отрицательных значений столбцов
print('Отрицательных значения в столбце days_employed:', df[df['days_employed'] < 0]['days_employed'].count())
print('Отрицательных значения в столбце dob_years:', df[df['dob_years'] < 0]['dob_years'].count())
print('Отрицательных значения в столбце total_income:', df[df['total_income'] < 0]['total_income'].count())

Отрицательных значения в столбце children: 47
Отрицательных значения в столбце days_employed: 15906
Отрицательных значения в столбце dob_years: 0
Отрицательных значения в столбце total_income: 0


В столбце `children` так же есть отрицательные значения. Переведем их в положительные. Для этого перезапишем все ячейки столбцов `days_employed` и `children` с применением функции abs(), которая вычисляет модуль числа.

In [5]:
df['days_employed'] = abs(df['days_employed']) # вычисляем модуль всех значений столбца 'days_employed' и перезаписываем их
df['children'] = abs(df['children']) # вычисляем модуль всех значений столбца 'children' и перезаписываем их
display(df.head()) # проверяем отсутствие отрицательных значений

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В столбце `children` проверим были ли отрицательные значения. Не исключено, что в нем также еще содержатся аномалии. Проверим уникальные значения столбца `children`.

In [6]:
df['children'].value_counts() # проверяем все уникальные значения

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

В 76 ячейках столбца указано количество детей - 20. Выведем эти строки на экран, чтобы изучить подробнее.

In [7]:
display(df[df['children'] == 20])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,NaN,покупка жилой недвижимости
21404,20,494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


Это аномалия, так как клиент с индексом 606 в возрасте 21 год не может иметь 20 детей. При изучении данных была найдена еще одна аномалия: в ячейке с возрастом указано нулевое значение. Эту аномалию изучем дальше, а пока решим, что делать с аномалией столбца `children`.

Вероятно, это человеская ошибка при вводе данных, 2 и 0 находятся рядом на числовом блоке клавиатуры. Так как одно из заданий заключается в изучении зависимости между количеством детей и возвратом кредита в срок, то заменим число детей на 2. Таких строк - 76, что составляет менее 1% от всех данных. Замена значений существенно не повлияет на результаты исследования. 

In [8]:
df.loc[df['children'] == 20, 'children'] = 2 # с помощью логической индексации меняем количество детей с 20 на 2
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Сейчас данные отображаются корректно. Теперь вернемся к аномалии с нулевым значениям возраста. Посчитаем количество таких строк.

In [9]:
df[df['dob_years'] == 0]['dob_years'].count() # счетчик ячеек с нулевым значением

101

Найдена 101 ячейка с нулевым значением. Это менее 1% от всех данных и,
так как в задании нет исследования зависимости возврата кредита от возраста, то оставим нулевые ячейки без изменений.

Вернемся к аномалии в столце `days_employed`. В столбце есть огромные значения трудового стажа, посчитаем количство этих строк. Возьмем за критерий расчета максимально возможный стаж. Для мужчин и женщин средний срок выхода на пенсию составляет 62,5 года. Так как по ТК РФ работать можно с 16 лет, то примем за максимально возможное значение стажа - 46,5 лет, то есть 16973 дней. 

In [10]:
df[df['days_employed'] > 16973]['days_employed'].count() # счетчик ячеек с огромным стажем

3447

Количество строк 3447, что составляет 16% датасета. Это существенный объем данных и удаление целых строк может повлиять на результаты исследования. Возможно, в ячейках указано количество часов стажа, а не дней. Проверим эту гипотезу. Для начала изучим эти строки.

In [11]:
display(df[df['days_employed'] > 16973].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
35,0,394021.072184,68,среднее,1,гражданский брак,1,M,пенсионер,0,77805.677436,на проведение свадьбы
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили
56,0,370145.087237,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,149141.043533,образование
71,0,338113.529892,62,среднее,1,женат / замужем,0,F,пенсионер,0,43929.696397,автомобили
78,0,359722.945074,61,высшее,0,женат / замужем,0,M,пенсионер,0,175127.646000,сделка с автомобилем


Можно заметить, что возраст клиентов - 50 лет и больше. Найдем средний возраст клиентов и среднее значение трудового стажа этих клиентов.

In [12]:
print(df[df['days_employed'] > 16973]['dob_years'].mean()) # вычисление среднего возраста
print(df[df['days_employed'] > 16973]['days_employed'].mean()) # вычисление среднего стажа, предположительно в часах

59.12590658543661
364802.9742311319


Средний возраст составляет 59 лет, по ТК РФ работать можно с 16 лет, значит у этих клиентов в среднем 43 года трудого стажа, или 15695 дней. 

Среднее значение трудового стажа составляет 364802. Если значения были указаны в часах, то 364802 / 24 получится 15200 дней.

Значения 15695 и 15200 достаточно близки, поэтому гипотеза подтвердилась. Переведем данные в ячейках из часов в дни.


In [13]:
df.loc[df['days_employed'] > 16973, 'days_employed'] = df.loc[df['days_employed'] > 16973, 'days_employed'] / 24 # перевод стажа из часов в дни

Для проверки результата работы функции, найдем максимальное значения стажа в днях.

In [14]:
df['days_employed'].max() # поиск максимального значения

16739.80835313875

Теперь максимальные значения не превышают 16739 дней, что соответствует стажу 45 лет. На этом закончим проверку данных на аномалии.

### Шаг 2.1 Заполнение пропусков

Найдем пропущенные значения в столбцах и посчитаем их количество.

In [15]:
df.isna().sum() # поиск пропущенных значений пропусков в ячейках

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В обоих столбцах по 2174 пропущенному значению с типом NaN, так как отсутствующие значения принадлежат к типу float.

Возможные причины пропусков в данных для столбцов `days_employed`(общий трудовой стаж в днях) и `total_income`(ежемесячный доход):
* У клиентов отсутствует трудовой стаж
* У клиентов отсутствует ежемесячный доход

Доля пропущенных значений для каждого столбца с пропусками составялет 10%. Это существенный объем данных и удаление целых строк с пустыми ячейками может повлиять на результаты исследования.

Заменим пустые ячейки медианными значениями в зависимости от группы при помощи метода transform(). Для количественных переменных это более объективно, чем использование среднего значения. Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства.


In [16]:
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median')) # замена пропусков медианным значением
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median')) 
df.isna().sum() # проверка отсутствия пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце `total_income` на целочисленный для удобства восприяти данных.

In [17]:
df['total_income'] = df['total_income'].astype('int') # изменение типа ячеек на целочисленный

### Шаг 2.4. Удаление дубликатов.

Проверим датасет на наличие явных дубликатов.

In [18]:
df.duplicated().sum() # подсчет явных дубликатов

54

Найдено 54 явных дубликата. Удалим их методом drop_duplicates() со сбросом индекса.

In [19]:
df = df.drop_duplicates().reset_index(drop=True)  # удаление явных дубликатов

Проверим наличие неявных дубликатов в столбце `education`.

In [20]:
df['education'].value_counts() # поиск уникальных значений

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

В столбце присутствует множество неявных дубликатов, с отличающимся регистром. Приведем все ячейки к нижнему регистру методом str.lower() и снова проверим датасет на наличие явных дубликатов.

In [21]:
df['education'] = df['education'].str.lower() # изменения регистра на строчный
df.duplicated().sum()

17

Найдено еще 17 явных дубликатов, так же избавимся от них методом drop_duplicates().

In [22]:
df = df.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов

Проверим наличие неявных дубликатов в столбце `family_status`.

In [23]:
df['family_status'].value_counts() # поиск уникальных значений

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

В стоблце `family_status` нет неявных дубликатов, но все равно приведем все ячейки к нижнему регистру для удобства чтения данных.

In [24]:
df['family_status'] = df['family_status'].str.lower()  # изменения регистра на строчный

Проверим наличие неявных дубликатов в столбце `income_type`.

In [25]:
df['income_type'].value_counts() # поиск уникальных значений

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

В стоблце `income_type` нет неявных дубликатов, оставим его без изменений.

Проверим наличие неявных дубликатов в столбце `purpose`.

In [26]:
df['purpose'].value_counts() # поиск уникальных значений

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Здесь достаточно много неявных дубликатов. Цели сформулированы по разному, но если их изучить, то можно выделить всего 4 категории целей:
* операции с автомобилем
* операции с недвижимостью
* проведение свадьбы
* получение образования

Позже напишем функцию, которая проверит все ячейки столбца `purpose` и сформирует новый столбец с соответствующими категориями.

На этом закончим этап поиска и удаления дубликатов. Для поиска дубликатов был использован метод value_counts(), который анализирует столбец, выбирает каждое уникальное значение и подсчитывает частоту его встречаемости в списке. Этот способ более эффективный чем duplicated().sum(), так как применяется только один метод, а не цепочка методов, что экономит ресурсы памяти компьютера.

Были найдены неявные дубликаты в столбцах с категориальными переменными. Возможно это произошло из-за того, что данные заполнялись разными сотрудниками банка,то есть имеет место человеческий фактор.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма со столбцами:
1. `education_id` и `education`
2. `family_status_id` и `family_status`

Получится два словаря с большим количеством дубликатов. Их нужно удалить с помощью цепочки методов: drop_duplicates() и reset_index().

In [27]:
edu_dict = df[['education_id', 'education']] # создания новых датафреймов
fam_dict = df[['family_status_id', 'family_status']]

fam_dict = fam_dict.drop_duplicates().reset_index(drop=True) # удаление дубликатов из датафреймов
edu_dict = edu_dict.drop_duplicates().reset_index(drop=True)

Удалим из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`.

In [28]:
df = df.drop(columns = ['education', 'family_status'], axis = 1) # удаление столбцов датасета после создания двух новых

### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создадим столбец `total_income_category` с категориями:
* 0–30000 — `E`
* 30001–50000 — `D`
* 50001–200000 — `C`
* 200001–1000000 — `B`
* 1000001 и выше — `A`

Например, кредитополучателю с доходом 25000 нужно назначить категорию `E`, а клиенту, получающему 235000, — `B`.

Для этого напишем функцию с условными конструкциями. Создадим новый столбец с применением метода apply() к столбцу `total_income`.

In [29]:
def income_category(income): # функция для присвоения категории в зависимости от уровня дохода
    if income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    return 'A'

df['total_income_category'] = df['total_income'].apply(income_category) # создание нового столбца с категорими и применения метода apply с функцией

Проверим новый столбец и корректность применения функции. Выведем первые 5 строк датасета.

In [30]:
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


Также создадим словарь, который будет использоваться в проверке гипотез для удобства восприятия информации.

In [31]:
income_dict = [                   
    ['A', 'более 1 000 000 руб.'], 
    ['B', '200 000 - 1 000 000 руб.'],
    ['C', '50 000 - 200 000 руб.'],
    ['D', '30 000 - 50 000 руб.'],
    ['E', 'менее 30 000 руб.']
]

columns=['total_income_category', 'income']
inc_dict = pd.DataFrame(data=income_dict, columns=columns) # созадние датафрейма для категорий уровня дохода

### Шаг 2.7. Категоризация целей кредита.

Напишем функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:
* операции с автомобилем
* операции с недвижимостью
* проведение свадьбы
* получение образования

In [32]:
def purpose_cat(purp): # функция для категоризации целей кредита
    try:
        if 'автомобил' in purp:
            return 'операции с автомобилем'
        if 'недвижимост' in purp:
            return 'операции с недвижимостью'
        if 'свадьб' in purp:
            return 'проведение свадьбы'
        if 'образовани' in purp:
            return 'получение образования'
    except:
        return 'цель не определена'

df['purpose_category'] = df['purpose'].apply(purpose_cat) # создание нового столбца с категорией в зависимости от цели кредита

Проверим результат работы функции и выведем на экран первые 5 строк датасета.

In [33]:
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,None
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,None
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для ответа на этот вопрос, построим сводную таблицу и проверим как количество детей в семье повлияло на наличие задолженности по возврату кредитов. Добавим столбец `ratio`, в котором будет расчитываться отношение должников ко всем клиентам по каждой группе клиентов. Выведем таблицу на экран.

In [34]:
df_pivot_children = df.pivot_table(index=['children'], columns='debt', values='dob_years', aggfunc='count') # сводная таблица зависимости наличия
df_pivot_children['ratio'] = round(df_pivot_children[1] / (df_pivot_children[0] + df_pivot_children[1]) * 100, 1)
display(df_pivot_children)

debt,0,1,ratio
children,,,
0,13028.0,1063.0,7.5
1,4410.0,445.0,9.2
2,1926.0,202.0,9.5
3,303.0,27.0,8.2
4,37.0,4.0,9.8
5,9.0,NaN,NaN


Распечатаем данные столбца `ratio` для дальнейшего вывода

In [35]:
print('Процент клиентов с задолженностью:\n') 
print('Семьи без детей:', df_pivot_children.loc[0, 'ratio'], '%')
print('Семьи с 1 ребенком:', df_pivot_children.loc[1,'ratio'], '%')
print('Семьи с 2 детьми:', df_pivot_children.loc[2,'ratio'], '%')
print('Семьи с 3 детьми:', df_pivot_children.loc[3,'ratio'], '%')
print('Семьи с 4 детьми:', df_pivot_children.loc[4,'ratio'], '%')

Процент клиентов с задолженностью:

Семьи без детей: 7.5 %
Семьи с 1 ребенком: 9.2 %
Семьи с 2 детьми: 9.5 %
Семьи с 3 детьми: 8.2 %
Семьи с 4 детьми: 9.8 %


##### Вывод 1:

Количество детей влияет на возврат кредита в срок. У клиентов без детей меньше всего задолженностей по кредиту - 7.5%. Доля задолженностей увеличивается почти на 2% у клиентов с одним ребенком. С увеличением количества детей - растет процент задолженностей, но незначительно в пределах одного процента. Исключение составляют клиенты с 3 детьми, процент задолженностей снижается до 8.2%. Максимум составляет 9.8% у клиентов с 4 детьми. У клиентов с 5 детьми нет задолженностей.

Клиенты без детей берут примерно в 2 раза больше кредитов, чем клиенты с детьми и имеют меньше всего задолженностей.

Наибольшее количество задолженностей у клиентов с 4 детьми.

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на этот вопрос, построим сводную таблицу и проверим как семейное положение повлияло на наличие задолженности по возврату кредитов. Для наглядности объединим таблицу со словарем `fam_dict`. Добавим столбец `ratio`, в котором будет расчитываться отношение должников ко всем клиентам по каждой группе клиентов. Выведем таблицу на экран.

In [36]:
df_pivot_family = df.pivot_table(index=['family_status_id'], columns='debt', values='dob_years', aggfunc='count') # сводная таблица зависимости наличия задолженности от семейного положения 
df_pivot_family = df_pivot_family.merge(fam_dict, on='family_status_id', how='left') # объединение сводной таблицы и "словаря"
df_pivot_family['ratio'] = round(df_pivot_family[1] / (df_pivot_family[0] + df_pivot_family[1]) * 100, 1)
display(df_pivot_family)

,family_status_id,0,1,family_status,ratio
0,0,11408,931,женат / замужем,7.5
1,1,3763,388,гражданский брак,9.3
2,2,896,63,вдовец / вдова,6.6
3,3,1110,85,в разводе,7.1
4,4,2536,274,не женат / не замужем,9.8


Распечатаем данные столбца `ratio` для дальнейшего вывода

In [37]:
print('Процент клиентов с задолженностью:\n') # вычисления отношения клиентов с задолженностью к клиентам без задолженности
print('женат / замужем:', df_pivot_family.loc[0, 'ratio'], '%')
print('гражданский брак:', df_pivot_family.loc[1, 'ratio'], '%')
print('не женат / не замужем:', df_pivot_family.loc[4, 'ratio'], '%')
print('в разводе:', df_pivot_family.loc[3, 'ratio'], '%')
print('вдовец / вдова:', df_pivot_family.loc[2, 'ratio'], '%')

Процент клиентов с задолженностью:

женат / замужем: 7.5 %
гражданский брак: 9.3 %
не женат / не замужем: 9.8 %
в разводе: 7.1 %
вдовец / вдова: 6.6 %


##### Вывод 2:

Семейное положение влияет на возврат кредита в срок. Самый низкий процент задолженностей у вдовцов и 6.6%, у клиентов в разводе 7.1%. У клиентов, состоящих в неофициальных отношениях процент задолженностей выше.  У клиентов в гражданском браке - 9.3%, максимальный процент тех, то не женат и не замужем - 9.8%. При вступленнии в официальный брак, процент задолженностей снижается до 7.5%.

Клиенты, состоящие в официальном браке больше других клиентов берут кредиты и имеют меньше задолженностей.

Максимальное количество задолженностей у клиентов, которые не женаты и не замужем.


##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на этот вопрос, построим сводную таблицу и проверим как уровень дохода повлиял на наличие задолженности по возврату кредитов. Для наглядности объединим таблицу со словарем `fam_inc`. Добавим столбец `ratio`, в котором будет расчитываться отношение должников ко всем клиентам по каждой группе клиентов. Выведем таблицу на экран. 

In [38]:
df_pivot_income = df.pivot_table(index=['total_income_category'], columns='debt', values='dob_years', aggfunc='count') # сводная таблица зависимости наличия задолженности от уровня дохода
df_pivot_income = df_pivot_income.merge(inc_dict, on='total_income_category', how='left') # объединение сводной таблицы и "словаря"
df_pivot_income['ratio'] = round(df_pivot_income[1] / (df_pivot_income[0] + df_pivot_income[1])  * 100, 1)
display(df_pivot_income)

,total_income_category,0,1,income,ratio
0,A,23,2,более 1 000 000 руб.,8.0
1,B,4686,356,200 000 - 1 000 000 руб.,7.1
2,C,14655,1360,50 000 - 200 000 руб.,8.5
3,D,329,21,30 000 - 50 000 руб.,6.0
4,E,20,2,менее 30 000 руб.,9.1


Распечатаем данные столбца `ratio` для дальнейшего вывода

In [39]:
print('Процент клиентов с задолженностью:\n') # вычисления отношения клиентов с задолженностью к клиентам без задолженности
print('A:', df_pivot_income.loc[0, 'ratio'], '%')
print('B:', df_pivot_income.loc[1, 'ratio'], '%')
print('C:', df_pivot_income.loc[2, 'ratio'], '%')
print('D:', df_pivot_income.loc[3, 'ratio'], '%')
print('E:', df_pivot_income.loc[4, 'ratio'], '%')

Процент клиентов с задолженностью:

A: 8.0 %
B: 7.1 %
C: 8.5 %
D: 6.0 %
E: 9.1 %


##### Вывод 3:

Уровень дохода влияет на возврат кредита в срок. Максимальная доля задолженностей - 9.1%, у клиентов с низким доходом (до 30 000 руб.) Минимум задолженностей - 6.0%, у клиентов с доходом от 30 000 руб. до 50 000 руб. Далее доля увеличивается до 8.5% с увеличением дохода до 200 000 руб. Доля снова падает до 7.1% при увеличении дохода до 1 000 000 руб. При доходе клиента более 1 000 000 руб. - доля задолженностей снова растет до 8.0%.

Максимальная вероятность к просрочке у группы с доходом до 30 000 руб.

Минимальная вероятность к просрочке у группы с доходом 30 000 до 50 000 руб.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

Для ответа на этот вопрос, построим сводную таблицу и проверим как цели кредита повлияли на наличие задолженности по возврату кредитов. Добавим столбец `ratio`, в котором будет расчитываться отношение должников ко всем клиентам по каждой группе клиентов. Выведем таблицу на экран. 

In [40]:
df_pivot_purpose = df.pivot_table(index=['purpose_category'], columns='debt', values='dob_years', aggfunc='count') # сводная таблица зависимости наличия задолженности от цели кредита
df_pivot_purpose['ratio'] = round(df_pivot_purpose[1] / (df_pivot_purpose[0] + df_pivot_purpose[1]) * 100, 1)
display(df_pivot_purpose)

debt,0,1,ratio
purpose_category,,,
операции с автомобилем,3903,403,9.4
операции с недвижимостью,5877,474,7.5
получение образования,3643,370,9.2
проведение свадьбы,2138,186,8.0


Распечатаем данные столбца `ratio` для дальнейшего вывода

In [41]:
print('Процент клиентов с задолженностью:\n') # вычисления отношения клиентов с задолженностью к клиентам без задолженности
print('операции с автомобилем:', df_pivot_purpose['ratio'][0], '%')
print('операции с недвижимостью:', df_pivot_purpose['ratio'][1], '%')
print('получение образования:', df_pivot_purpose['ratio'][2], '%')
print('проведение свадьбы:', df_pivot_purpose['ratio'][3], '%')

Процент клиентов с задолженностью:

операции с автомобилем: 9.4 %
операции с недвижимостью: 7.5 %
получение образования: 9.2 %
проведение свадьбы: 8.0 %


##### Вывод 4:

Цель кредита влияет на возврат кредита в срок. Самый низкий процент задолженностей у клиентов с кредитами на операции с недвижимостью - 7.5%. Самые высокие проценты задолженностей у клиентов с кредитами на получение образования - 9.2% и кредитами на операции с автомобилем - 9.4%. На проведение свадеб приходится - 8% клиентов с задолженностями.

Больше кредитов приходится на операции с недвижимостью и клиенты этой категории имеют меньше всего задолженностей.

Максимальное количество задолженностей у клиентов с кредитами на операции с автомобилем.

## Общий вывод:

Было проверено влияют ли на срок возврата кредита следующие факторы:
* количество детей
* семейное положение
* уровень дохода
* цель кредита

Во всех случаях доля клиентов с задолженностями не превышает 10% от числа клиентов в конкретной категории, а изменения между категориями в каждой исселедуемой категории не превышают 4%.

Так как количество детей и уровень дохода являются количественными переменными, то здесь можно оценить динамику изменений. С увеличением количества детей растет процент задолженностей, но незначительно. Более высокий рост задолженностей наблюдается при появлении первого ребенка. При увеличении уровня дохода можно заметить сначала уменьшение количества задолженностей, затем увеличение, но общая динамика показывает снижение.

Семейное положение и цель кредита являются категориальными переменными, поэтому здесь можно выделить только некоторые особенности. Клиенты, состоящие в официальном браке больше других клиентов берут кредиты и имеют меньше задолженностей. Больше всего кредитов приходится на операции с недвижимостью и клиенты этой категории имеют меньше всего задолженностей.